<a href="https://colab.research.google.com/github/NavedAFZ/neural-networks/blob/master/bianry_class_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from numpy import vstack
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

In [0]:
class CSV_data(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = read_csv(path, header=None)
        # store the inputs and outputs
        self.X = df.values[:, :-1]
        self.y = df.values[:, -1]
        # ensure input data is floats
        self.X = self.X.astype('float32')
        # label encode target and ensure the values are floats
        self.y = LabelEncoder().fit_transform(self.y)
        self.y = self.y.astype('float32')
        self.y = self.y.reshape((len(self.y), 1))
 
    # number of rows in the dataset
    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
 
    # get indexes for train and test rows
    def __split__(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])


In [3]:
pip install notebook --upgrade

     |████████████████████████████████| 9.7MB 2.7MB/s 
     |████████████████████████████████| 501kB 46.0MB/s 
ERROR: Operation cancelled by user


In [0]:
class my_model(Module):
  def __init__(self,n_inputs):
    super(my_model,self).__init__()
    self.hidden1=Linear(n_inputs,10)
    kaiming_uniform_(self.hidden1.weight,nonlinearity='relu')
    self.act1=ReLU()
    self.hidden2=Linear(10,8)
    kaiming_uniform_(self.hidden2.weight,nonlinearity='relu')
    self.act2=ReLU()
    self.hidden3=Linear(8,1)
    xavier_uniform_(self.hidden3.weight)
    self.act3=Sigmoid()
  def forward(self,X):
    X=self.hidden1(X)
    X=self.act1(X)
    X=self.hidden2(X)
    X=self.act2(X)
    X=self.hidden3(X)
    X=self.act3(X)
    return X
      


In [0]:
"""def prepare_data(path):
  dataset=CSV_data(path)
  train,test=dataset.__split__()
  train_loader=DataLoader(train,batch_size=32,shuffle=True)
  test_loader=DataLoader(test,batch_size=1024,shuffle=False)
  return train_loader,test_loader"""
def prepare_data(path):
    # load the dataset
    dataset = CSV_data(path)
    # calculate split
    train, test = dataset.__split__()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl  

In [0]:
"""def train_model(train_l,model):
  loss_fn=BCELoss()
  opt=SGD(model.parameters(),lr=0.1,momentum=0.9)
  for epoch in range(100):
    for i, (train_d,targets) in enumerate(train_l):
      opt.zero_grad()
      out=model(train_d)
      loss=loss_fn(out,target)
      loss.backward()
      opt.step()"""
def train_model(train_dl, model):
    # define the optimization
    criterion = BCELoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in range(100):
        # enumerate mini batches
        for  data in train_dl:
          inputs, targets=data          
          # clear the gradients
          optimizer.zero_grad()
          # compute the model output
          yhat = model(inputs)
          # calculate loss
          loss = criterion(yhat, targets)
          # credit assignment
          loss.backward()
          # update model weights
          optimizer.step()      



In [0]:
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # round to class values
        yhat = yhat.round()
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    return acc


In [0]:
def predict(model,data):
  data1=Tensor([data])
  out=model(data1)
  out=out.detach().numpy()
  return out

In [9]:
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
train11,test11=prepare_data(path)
print(len(train11.dataset), len(test11.dataset))
mod1=my_model(34)

train_model(train11,mod1)
acc=evaluate_model(test11,mod1)
print(acc)


235 116
0.9224137931034483


In [10]:
print(mod1)

my_model(
  (hidden1): Linear(in_features=34, out_features=10, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=10, out_features=8, bias=True)
  (act2): ReLU()
  (hidden3): Linear(in_features=8, out_features=1, bias=True)
  (act3): Sigmoid()
)
